In [1]:
# Importa Pandas e find_best_hyperparameter
import pandas as pd
from find_best_hyperparameters import AtividadeTres

# Carrega Atividade 3
a3 = AtividadeTres(26)

# Define hiper parâmetros para Greedy Search
hyper_params = {}

hyper_params['nb'] = {
    'model_name': 'MultinomialNB',
    'params': a3.create_param_grid({   
        # Explicar alpha
        'alpha': [0.1, 0.25, 0.5, 0.75, 1.0, 2.0, 5.0],
        # Aprender probabilidades anteriores de classe
        'fit_prior': [True, False]
    })
}

hyper_params['lr'] = {
    'model_name': 'LogisticRegression',
    'params': a3.create_param_grid({
        # Força da regularização
        'C': [0.1, 0.25, 1.0, 5.0, 10.0, 20.0],
        # Algoritmos de otimização
        'solver': ['lbfgs', 'liblinear', 'sag'],
        # Tipos de regularização
        'penalty': ['l1', 'l2', 'elasticnet', None],
        # Iterações máximas
        'max_iter': [100, 200, 500, 1000]
    })
}

hyper_params['svm'] = {
    'model_name': 'LinearSVC',
    'params': a3.create_param_grid({
        # Força da regularização
        'C': [0.01, 0.1, 1, 5, 10, 50, 100],
        # Função de perda
        'loss': ['hinge', 'squared_hinge'],
        # Iterações máximas
        'max_iter': [100, 250, 500, 1000, 5000],
        # Critério de tolerância para parar
        'tol': [1e-3, 1e-4, 1e-5]
    })
}

Classe iniciada com semente 26!
Hiper parâmetros montados: dict_keys(['alpha', 'fit_prior'])
Hiper parâmetros montados: dict_keys(['C', 'solver', 'penalty', 'max_iter'])
Hiper parâmetros montados: dict_keys(['C', 'loss', 'max_iter', 'tol'])


In [2]:
# Leitura e processamento da base CSTR
BASE='CSTR'
FILENAME='CSTR.csv'
cstr = a3.load_data(FILENAME)
cstr.head()

,file_name,text,class
0,126.txt,Rhetorical (Rhet) is a programming / knowledge...,ArtificiallIntelligence
1,5.txt,Reduction is the operation of transforming a p...,ArtificiallIntelligence
2,48.txt,"For years, researchers have used knowledge-int...",ArtificiallIntelligence
3,81.txt,Proceedings of a workshop held in conjunction ...,ArtificiallIntelligence
4,25.txt,The Medication Advisor is the latest project o...,ArtificiallIntelligence


In [3]:
# Separação dos dados de treino e teste
train, test, series = a3.split_data(data=cstr, text_col='text', class_col='class', test_size=0.2)

# Salva resultados
a3.save_data_frame(train, BASE + '_train.csv')
a3.save_data_frame(test, BASE + '_test.csv')

# Mostra primeiras linhas do teste
test.head()

Dados divididos com tamanho do teste 0.20 e semente 26!
Dicionário de séries: dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])
Arquivo out/CSTR_train.csv salvo com sucesso!
Arquivo out/CSTR_test.csv salvo com sucesso!


,text,class
212,In this paper we present a system for vision-b...,Robotics
72,"WordNet is a lexical database that, among othe...",ArtificiallIntelligence
172,In this paper we introduce a formalism for opt...,Robotics
136,Predictive coding and temporal invariance are ...,Robotics
135,A selective vision system sequentially collect...,Robotics


In [4]:
# Greedy Search para NB, LR and SVM
best_greedy = {}
greedy = None

for item in hyper_params:
    # Testa parâmetros com Greedy Search
    greedy = pd.DataFrame(a3.greedy_search(
        model_name=hyper_params[item]['model_name'],
        param_grid=hyper_params[item]['params'],
        series=series
    ))
    # Salva resultados em arquivo
    a3.save_data_frame(greedy, BASE + '_greedy_' + item + '.csv')
    # Resgata melhor resultado para treinamento
    max_score = greedy['score'].max()
    max_rows = greedy[greedy['score'] == max_score]
    best_greedy[item] = max_rows.sample(1).reset_index(drop=True).iloc[0].to_dict()
    
# Exibe valores no notebook
pd.DataFrame(best_greedy)

Arquivo out/CSTR_greedy_nb.csv salvo com sucesso!
Arquivo out/CSTR_greedy_lr.csv salvo com sucesso!
Arquivo out/CSTR_greedy_svm.csv salvo com sucesso!


,nb,lr,svm
alpha,1.0,NaN,NaN
fit_prior,True,NaN,NaN
score,0.974895,0.974895,0.974895
C,NaN,1.0,10.0
solver,NaN,liblinear,NaN
penalty,NaN,l2,NaN
max_iter,NaN,200,1000
loss,NaN,NaN,squared_hinge
tol,NaN,NaN,0.001


In [5]:
# Treina modelo com melhores parâmetros
metrics = {}

for item in best_greedy:
    # Remove key score
    try:
        best_greedy[item].pop('score')
    except:
        pass
    metrics[item] = a3.train(model_name=item, params=best_greedy[item], series=series, base=BASE)

a3.save_data_frame(pd.DataFrame(metrics), BASE + '_metrics.csv')

# Exibe valores no notebook
pd.DataFrame(metrics)

Arquivo out/CSTR_metrics.csv salvo com sucesso!


,nb,lr,svm
f1_score_macro,0.800952,0.835279,0.855055
f1_score_micro,0.833333,0.800000,0.833333
accuracy,0.833333,0.800000,0.833333


In [1]:
# Agora tudo em uma única célula para facilitar processar nova base

# Configure sua base para processar
BASE='DmozComp'
FILENAME='Dmoz-Computers.csv'

# Aperte o play, sente e relaxe
import pandas as pd
from find_best_hyperparameters import AtividadeTres

a3 = AtividadeTres(26)
cstr = a3.load_data(FILENAME)

hyper_params = {}
hyper_params['nb'] = {'model_name': 'MultinomialNB','params': a3.create_param_grid({   
        'alpha': [0.1, 0.25, 0.5, 0.75, 1.0, 2.0, 5.0],
        'fit_prior': [True, False]
    })}
hyper_params['lr'] = {'model_name': 'LogisticRegression','params': a3.create_param_grid({
        'C': [0.1, 0.25, 1.0, 5.0, 10.0, 20.0],
        'solver': ['lbfgs', 'liblinear', 'sag'],
        'penalty': ['l1', 'l2', 'elasticnet', None],
        'max_iter': [100, 200, 500, 1000]
    })}
hyper_params['svm'] = {'model_name': 'LinearSVC','params': a3.create_param_grid({
        'C': [0.01, 0.1, 1, 5, 10, 50, 100],
        'loss': ['hinge', 'squared_hinge'],
        'max_iter': [100, 250, 500, 1000, 5000],
        'tol': [1e-3, 1e-4, 1e-5]
    })}

train, test, series = a3.split_data(data=cstr, text_col='text', class_col='class', test_size=0.2)
a3.save_data_frame(train, BASE + '_train.csv')
a3.save_data_frame(test, BASE + '_test.csv')

best_greedy = {}
greedy = None
for item in hyper_params:
    greedy = pd.DataFrame(a3.greedy_search(
        model_name=hyper_params[item]['model_name'],
        param_grid=hyper_params[item]['params'],
        series=series
    ))
    a3.save_data_frame(greedy, BASE + '_greedy_' + item + '.csv')
    max_score = greedy['score'].max()
    max_rows = greedy[greedy['score'] == max_score]
    best_greedy[item] = max_rows.sample(1).reset_index(drop=True).iloc[0].to_dict()

metrics = {}
for item in best_greedy:
    try:
        best_greedy[item].pop('score')
    except:
        pass
    metrics[item] = a3.train(model_name=item, params=best_greedy[item], series=series, base=BASE)
a3.save_data_frame(pd.DataFrame(metrics), BASE + '_metrics.csv')

Classe iniciada com semente 26!
Hiper parâmetros montados: dict_keys(['alpha', 'fit_prior'])
Hiper parâmetros montados: dict_keys(['C', 'solver', 'penalty', 'max_iter'])
Hiper parâmetros montados: dict_keys(['C', 'loss', 'max_iter', 'tol'])
Dados divididos com tamanho do teste 0.20 e semente 26!
Dicionário de séries: dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])
Arquivo out/DmozComp_train.csv salvo com sucesso!
Arquivo out/DmozComp_test.csv salvo com sucesso!
Arquivo out/DmozComp_greedy_nb.csv salvo com sucesso!
Arquivo out/DmozComp_greedy_lr.csv salvo com sucesso!
Arquivo out/DmozComp_greedy_svm.csv salvo com sucesso!
Arquivo out/DmozComp_metrics.csv salvo com sucesso!
